Last updated: 16 Feb 2023

# 👋 PyCaret Time Series Forecasting Tutorial

PyCaret is an open-source, low-code machine learning library in Python that automates machine learning workflows. It is an end-to-end machine learning and model management tool that exponentially speeds up the experiment cycle and makes you more productive.

Compared with the other open-source machine learning libraries, PyCaret is an alternate low-code library that can be used to replace hundreds of lines of code with a few lines only. This makes experiments exponentially fast and efficient. PyCaret is essentially a Python wrapper around several machine learning libraries and frameworks, such as scikit-learn, XGBoost, LightGBM, CatBoost, spaCy, Optuna, Hyperopt, Ray, and a few more.

The design and simplicity of PyCaret are inspired by the emerging role of citizen data scientists, a term first used by Gartner. Citizen Data Scientists are power users who can perform both simple and moderately sophisticated analytical tasks that would previously have required more technical expertise.


# 💻 Installation

PyCaret is tested and supported on the following 64-bit systems:
- Python 3.7 – 3.10
- Python 3.9 for Ubuntu only
- Ubuntu 16.04 or later
- Windows 7 or later

You can install PyCaret with Python's pip package manager:

`pip install pycaret`

PyCaret's default installation will not install all the extra dependencies automatically. For that you will have to install the full version:

`pip install pycaret[full]`

or depending on your use-case you may install one of the following variant:

- `pip install pycaret[analysis]`
- `pip install pycaret[models]`
- `pip install pycaret[tuner]`
- `pip install pycaret[mlops]`
- `pip install pycaret[parallel]`
- `pip install pycaret[test]`

In [ ]:
# install pycaret
#!pip install pycaret[full]
# check installed version
import pycaret 
pycaret.__version__

# 🚀 Quick start

PyCaret's time series forecasting module is now available. The module currently is suitable for univariate / multivariate time series forecasting tasks. The API of time series module is consistent with other modules of PyCaret.

It comes built-in with preprocessing capabilities and over 30 algorithms comprising of statistical / time-series methods as well as machine learning based models. In addition to the model training, this module has lot of other capabilities such as automated hyperparameter tuning, ensembling, model analysis, model packaging and deployment capabilities. 

A typical workflow in PyCaret consist of following 5 steps in this order:

### **Setup** ➡️ **Compare Models** ➡️ **Analyze Model** ➡️ **Prediction** ➡️ **Save Model** <br/>

In [ ]:
# import pandas module
import pandas as pd
import re
import numpy as np

# global variables
Soil_humi = []
Resistance = []
Soil_temp = []
total_amount = 0
rolling_mean_window_data = 15
rolling_mean_window_grouped = 5
min_length_df = 400 #was 50
slope_irrigation = -0.07

# creating a data frame
data = pd.read_csv("binned_removed_new.csv",header=0)
data.head()

# create array with sensors strings
for col in data.columns:
    if re.findall(r"SoilHumiditySensor$", col):
        Soil_humi.append(col)
    elif re.findall(r"\/ $", col):
        Resistance.append(col)
    elif re.findall(r"SoilThermometer$", col):
        Soil_temp.append(col)

# print them to check
for i in range(len(Soil_humi)):
    total_amount += 1
    print(Soil_humi[i])
for i in range(len(Resistance)):
    total_amount += 1
    print(Resistance[i])
for i in range(len(Soil_temp)):
    total_amount += 1
    print(Soil_temp[i])
    
print("Total amount of values: ", total_amount)

In [ ]:
# plot the dataset
data.plot(figsize = (20,10))

In [ ]:
# Show if there are any missing values inside the data
data.isna().any()

In [ ]:
data.tail()

In [ ]:
# creating a data frame
data_weather = pd.read_csv("april_weather.csv", header=0, sep=";")
data_weather.head()

In [ ]:
# Create a sample DataFrame with a datetime column
df = pd.DataFrame({'datetime': ['01/04/2023 00:03:43', '01/04/2023 00:08:43', '01/04/2023 02:05:47']})

print("before: \n", df, "\n", df.dtypes, "\n")

# Convert the datetime column to a datetime object
df['datetime'] = pd.to_datetime(df['datetime'], format='%d/%m/%Y %H:%M:%S')

print("After datetime, before ISO:  \n", df, "\n", df.dtypes, "\n")

# Format the datetime object as an ISO time string
df['datetime_iso'] = df['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Print the resulting DataFrame
print("After ISO:  \n", df, "\n", df.dtypes, "\n")

In [ ]:
#print("before: \n", data_weather, "\n", data_weather.dtypes, "\n")
# convert the timestamp column to a datetime object
#data_weather['Heure'] = pd.to_datetime(data_weather['Heure'])

# Convert the datetime column to a datetime object
data_weather['Heure'] = pd.to_datetime(data_weather['Heure'], format=' %d/%m/%Y %H:%M:%S')#, errors='coerce')
#print("After datetime, before ISO:  \n", data_weather, "\n", data_weather.dtypes, "\n")

# Format the datetime object as an ISO time string
data_weather['Heure'] = data_weather['Heure'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Print the resulting DataFrame
#data_weather.head()
#print("After ISO:  \n", data_weather, "\n", data_weather.dtypes, "\n")

In [ ]:
# Drop useless
data_weather = data_weather.drop(['Température intérieure(°C)', 'Humidité intérieure(%)', 'Intervalle', 'NO.'], axis=1)

In [ ]:
data_weather = data_weather.rename(columns={'Heure':'Time'})
data_weather.head(20)

In [ ]:
data_weather['Time'] = pd.to_datetime(data_weather['Time'])
data_weather.dtypes

In [ ]:
# set the index of the DataFrame to the timestamp column
data_weather = data_weather.set_index('Time')

In [ ]:
data_weather.head()

In [ ]:
#ref_dtype = 'float64'
# iterate through the columns and compare their data types to the reference data type
#for col in data_weather.columns:
#    print("This is :",col)
#    if data_weather[col].dtype == 'datetime64[ns]':
#        data_weather[col] = data_weather[col].resample('10T').ffill()
#    if data_weather[col].dtype == ref_dtype:
#        data_weather[col] = data_weather[col].resample('10T').mean()
#    else:
#        data_weather[col] = data_weather[col].resample('10T').ffill()

In [ ]:
data_weather_re = data_weather.resample('10T').fillna(method = 'bfill') #TODO: fill with mean!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
data_weather_re.index

In [ ]:
data_weather_re.head(10)

In [ ]:
data['Time'] = data['Time'].str[:-9]
data['Time']=pd.to_datetime(data['Time'])
data = data.set_index('Time')

In [ ]:
data.head()

In [ ]:
# concatenate the DataFrames vertically
#data_comb = pd.concat([data, data_weather_re], axis=0)
data_comb = pd.merge(data, data_weather_re, left_index=True, right_index=True, how='outer')

# print the resulting DataFrame
data_comb.tail(10)

In [ ]:
import missingno as msno 
# Plot correlation heatmap of missingness
msno.matrix(data_comb)

In [ ]:
# Select the rows between two dates
start_date = str(data_weather_re.index[0])
end_date = str(data_weather_re.index[-1])
data_comb_cut = data_comb.loc[start_date:end_date]

data = data_comb_cut

In [ ]:
msno.matrix(data)

In [ ]:
data.head()

In [ ]:
# Convert wind direction to integer
directions = [" N", " NNE", " NE", " ENE", " E", " ESE", " SE", " SSE", " S", " SSW", " SW", " WSW", " W", " WNW", " NW", " NNW"]
target_col = 'Direction du vent'

for index, value in data[target_col].iteritems():
    # Calculate wind direction in degrees -> TODO : convert to int and round
    if value == " ---":
        data[target_col][index] = np.nan
    else:
        data[target_col][index] = directions.index(value) * 22.5
        
data[target_col] = data[target_col].astype("float64")

In [ ]:
data.dtypes

In [ ]:
ref_dtype = 'float64'
obj_dtype = 'object'

for col in data.columns:
    if data[col].dtype == ref_dtype:
        data[col] = data[col].rolling(window=rolling_mean_window_data, win_type='gaussian').mean(std=rolling_mean_window_data)
    elif data[col].dtype == obj_dtype:
        data[col] = pd.to_numeric(data[col], errors='coerce')
        
    print("This is :",col, "it has the following dtype: ", data[col].dtype)

In [ ]:
msno.heatmap(data)

In [ ]:
msno.dendrogram(data)

In [ ]:
data.isna().any()

In [ ]:
data.plot(subplots=True, figsize=(20,50))

We can use the following estimators:
-----------------------------------
LinearRegression: LinearRegression

BayesianRidge: Regularized linear regression.

DecisionTreeRegressor: Decision tree regression. => no

ExtraTreesRegressor: Extremely randomized trees regression. => slow

KNeighborsRegressor: k-Nearest Neighbors regression. => no

RandomForestRegressor: Random forest regression. => slow

Ridge: Linear least squares with l2 regularization. => best?

SVR: Support vector regression. => slow, no

In [ ]:
# TODO: compare them with orig data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Init the transformer
reg_imp = IterativeImputer(estimator=Ridge())

# Fit/transform
data_imp = pd.DataFrame(reg_imp.fit_transform(data), columns=data.columns)

In [ ]:
data_imp.plot(subplots=True, figsize=(20,50))

In [ ]:
data_imp.head()

In [ ]:
def compare_dataframes(df1, df2):
    # Verify column names
    if not set(df1.columns) == set(df2.columns):
        print("Column names are not the same in both dataframes.")
        return

    # Find the common columns between the two dataframes
    common_columns = df1.columns.intersection(df2.columns)

    # Compare the available values in the common columns
    for column in common_columns:
        values1 = df1[column].dropna()
        values2 = df2[column].dropna()

        min_length = min(len(values1), len(values2))
        values1 = values1[:min_length]
        values2 = values2[:min_length]

        diff = np.abs(values1.values - values2.values)
        mae = np.mean(diff)
        rmse = np.sqrt(np.mean(diff ** 2))
        mpe = np.mean(diff / values1.values) * 100

        print(f"Column: {column}")
        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MPE: {mpe:.2f}%")
        print("df1 len:",len(values1),"df2 len:",len(values2))
        print()

compare_dataframes(data, data_imp)

In [ ]:
data_imp.index = data.index
data = data_imp

In [ ]:
data_imp.plot(subplots=True, figsize=(20,50))

In [ ]:
#impute data
#from sklearn.impute import KNNImputer

##data_knn_imputed = data_knn_imputed.drop(['Time'], axis=1)

# Init the transformer
#knn_imp = KNNImputer(n_neighbors=10)

# Fit/transform
#data_imp.iloc[:,0:len(data)] = knn_imp.fit_transform(data_imp.iloc[:,0:len(data)])

In [ ]:
#data_imp.plot(subplots=True, figsize=(20,50))

In [ ]:
# Plot correlation heatmap of missingness
msno.matrix(data)

In [ ]:
# Show if there are any missing values inside the data
data.isna().any()

In [ ]:
# create average cols
data['grouped_soil'] = data[Soil_humi].mean(axis=1)
data['grouped_resistance'] = data[Resistance].mean(axis=1)
data['grouped_soil_temp'] = data[Soil_temp].mean(axis=1)

In [ ]:
data.index.dtype

In [ ]:
#data['Time'] = data['Time'].str[:-9]
#data.head()

In [ ]:
#data['Time']=pd.to_datetime(data['Time'])

#data.head()

In [ ]:
# after concat => need to convert to datetimeindex once more:
#data.index=pd.to_datetime(data.index, format='%Y-%m-%d %H:%M:%S') #2023-04-01 00:50:00

In [ ]:
## Create Features
## create rolling mean: introduces NaN again -> later just cut off
data['rolling_mean_grouped_soil'] = data['grouped_soil'].rolling(window=rolling_mean_window_grouped, win_type='gaussian').mean(std=rolling_mean_window_grouped)
data['rolling_mean_grouped_soil_temp'] = data['grouped_soil_temp'].rolling(window=rolling_mean_window_grouped, win_type='gaussian').mean(std=rolling_mean_window_grouped)

data['hour'] = data.index.hour#.astype("float64")
data['minute'] = data.index.minute#.astype("float64")
data['date'] = data.index.day#.astype("float64")
data['month'] = data.index.month#.astype("float64")
data['day_of_year'] = data.index.dayofyear#.astype("float64")

In [ ]:
# feature scaling
data.dtypes

In [ ]:
# Min-Max Normalization
df = data.drop(['rolling_mean_grouped_soil', 'hour', 'minute', 'date', 'month', 'day_of_year'], axis=1)
index = data.index
df_norm = pd.DataFrame()
for col in df.columns:
    print("Current col: ", col)
    if df[col].dtype == ref_dtype:
        print("Normalize:   ", col)
        df_norm[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())
df_norm = pd.concat([df_norm, data['hour'], data['minute'], data['date'], data['month'], data['day_of_year'], data.rolling_mean_grouped_soil], 1)
data = df_norm

In [ ]:
data.dtypes

In [ ]:
print(str(Soil_temp).replace("[", "").replace("]", ""))

In [ ]:
int(5/2 + 0.5)

In [ ]:
data.head()

In [ ]:
#data = data[['Time', 'hour', 'minute', 'date', 'month', 'grouped_soil', 'grouped_resistance', 'grouped_soil_temp', 'rolling_mean_grouped_soil', 'rolling_mean_grouped_soil_temp', 
#             str(Soil_humi).replace("[", "").replace("]", "").replace("\\",""),
#             str(Resistance).replace("[", "").replace("]", "").replace("\\",""),
#             str(Soil_temp).replace("[", "").replace("]", "").replace("\\","")
#            ]]

# drop those values without rolling_mean
#data = data[rolling_mean_window_data+int(rolling_mean_window_grouped/2 + 0.5):]
data = data[4:]

# reset changed index(due to drop)
#data = data.reset_index()

#data = data.drop(['index'],axis=1)

In [ ]:
print(data.iloc[0])

data.head(10)

In [ ]:
data.dtypes

In [ ]:
#data_plot = data.drop(['hour','minute','date','month','grouped_soil','grouped_resistance','grouped_soil_temp', Soil_humi, Resistance, Soil_temp], axis=1)
#data_plot.tail(20)

In [ ]:
#data_plot.set_index('Time').plot(figsize = (20,10))

In [ ]:
# Split dataset into train and test set
def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

train, test = split_data(data, pd.to_datetime('2023-04-25 15:30')) # splitting the data for training before 15th June

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.xlabel('Time')
plt.ylabel('rolling_mean_grouped_soil')
plt.plot(train.index,train['rolling_mean_grouped_soil'],label='train')
plt.plot(test.index,test['rolling_mean_grouped_soil'],label='test')
plt.legend()
plt.show()

## Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. `Setup` has only one required parameter i.e. `data`. All the other parameters are optional.

Once the setup has been successfully executed it shows the information grid containing experiment level information. 

- **Session id:**  A pseudo-random number distributed as a seed in all functions for later reproducibility. If no `session_id` is passed, a random number is automatically generated that is distributed to all functions.<br/>
<br/>
- **Approach:**  Univariate or multivariate. <br/>
<br/>
- **Exogenous Variables:**  Exogeneous variables to be used in model. <br/>
<br/>
- **Original data shape:**  Shape of the original data prior to any transformations. <br/>
<br/>
- **Transformed train set shape :**  Shape of transformed train set <br/>
<br/>
- **Transformed test set shape :**  Shape of transformed test set <br/>
<br/>

PyCaret has two set of API's that you can work with. (1) Functional (as seen above) and (2) Object Oriented API.

With Object Oriented API instead of executing functions directly you will import a class and execute methods of class.

In [ ]:
train.dtypes

In [ ]:
# import TSForecastingExperiment and init the class
#from pycaret.regression import *
from pycaret.time_series import *
exp=TSForecastingExperiment()
   
# init setup on exp
exp.setup(
    train, 
    target = 'rolling_mean_grouped_soil', 
    enforce_exogenous = False, 
    fold_strategy='sliding', 
    fh = 200, 
    session_id = 123, 
    fold = 3,
    ignore_features = ['orig_index', 'gradient',
                       '638de53168f31919048f189d; 63932c6e68f319085df375b5 / B2_solar_x2_03; Soil_tension / SoilHumiditySensor','638de53168f31919048f189d; 63932c6e68f319085df375b7 / B2_solar_x2_03; Resistance / ',
                       '638de53168f31919048f189d; 63932c6e68f319085df375b9 / B2_solar_x2_03; Soil_temperature / SoilThermometer', '638de56a68f31919048f189e; 63932ce468f319085df375c9 / B3_solar_x1_02; Soil_tension / SoilHumiditySensor',
                       '638de56a68f31919048f189e; 63932ce468f319085df375cb / B3_solar_x1_02; Resistance / ', '638de57568f31919048f189f; 63932c3668f319085df375ab / B4_solar_x1_03; Soil_tension / SoilHumiditySensor',
                       '638de57568f31919048f189f; 63932c3668f319085df375ad / B4_solar_x1_03; Resistance / ', '638de57568f31919048f189f; 63932c3668f319085df375af / B4_solar_x1_03; Soil_temperature / SoilThermometer']
)

In [ ]:
# show available models
exp.models()

You can use any of the two method i.e. Functional or OOP and even switch back and forth between two set of API's. The choice of method will not impact the results and has been tested for consistency.

## Check Stats
The `check_stats` function is used to get summary statistics and run statistical tests on the original data or model residuals.

In [ ]:
# check statistical tests on original data
exp.check_stats()

## Compare Models

This function trains and evaluates the performance of all the estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# compare baseline models
best= exp.compare_models(
    n_select = 15, 
    fold = 3, 
    sort = 'R2',# 
    verbose = 1#, 
    #exclude=['lar_cds_dt','auto_arima','arima'],
    #include=['lr_cds_dt', 'br_cds_dt', 'ridge_cds_dt', 
    #         'huber_cds_dt', 'knn_cds_dt', 'catboost_cds_dt']
)

In [ ]:
for i in range(len(best)):
    print("\n ", i, ". best model is:")
    print(best[i])

## Analyze Model

You can use the `plot_model` function to analyzes the performance of a trained model on the test set. It may require re-training the model in certain cases.

In [ ]:
# plot forecast
exp.plot_model(best, plot = 'forecast')

In [ ]:
# plot forecast for 36 months in future
for j in range(len(best)):
    print("This is the ", j,  ". best model")
    exp.plot_model(best[j], plot = 'forecast', data_kwargs = {'fh' : 750})

In [ ]:
# residuals plot
for i in range(len(best)):
    exp.plot_model(best[i], plot = 'residuals')

In [ ]:
# feature plot
#for i in range(len(best)):
#    exp.plot_model(best[i], plot = 'feature')

In [ ]:
# check docstring to see available plots 
#help(plot_model)

An alternate to `plot_model` function is `evaluate_model`. It can only be used in Notebook since it uses ipywidget.

## Prediction
The `predict_model` function returns `y_pred`. When data is `None` (default), it uses `fh` as defined during the `setup` function. 

In [ ]:
# predict on test set
holdout_pred = []
for i in range(len(best)):
    holdout_pred.append(exp.predict_model(best[i]))

In [ ]:
# show predictions df
for i in range(len(holdout_pred)):
    print(holdout_pred[i].head(30))

In [ ]:
# generate forecast for 36 period in future
for i in range(len(best)):
    print(exp.predict_model(best[i], fh = 36))

## Save Model

Finally, you can save the entire pipeline on disk for later use, using pycaret's `save_model` function.

In [ ]:
# save pipeline
for i in range(len(best)):
    exp.save_model(best[i], 'my_first_pipeline_TS_' + str(i))

In [ ]:
# load pipeline
loaded_best_pipeline = []
for i in range(len(best)): # TODO: exp will not work if it is not set before, use OS later
    loaded_best_pipeline.append(load_model('my_first_pipeline_TS_' + str(i)))
    loaded_best_pipeline

# 👇 Detailed function-by-function overview

## ✅ Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. `Setup` has only one required parameter i.e. `data`. All the other parameters are optional.

In [ ]:
s2 = []
for i in range(len(cut_sub_dfs)):
    s2.append(setup(
        cut_sub_dfs[i], 
        target = 'rolling_mean_grouped_soil', 
        fh = 50, 
        session_id = 123, 
        use_gpu = True, 
        ignore_features = ['Time', 'orig_index', 'gradient'],
        fold=3
    ))

To access all the variables created by the setup function such as transformed dataset, random_state, etc. you can use `get_config` method.

In [ ]:
# check all available config
get_config()

In [ ]:
# lets access y_train_transformed
get_config('y_train_transformed')

In [ ]:
# another example: let's access seed
print("The current seed is: {}".format(get_config('seed')))

# now lets change it using set_config
set_config('seed', 786)
print("The new seed is: {}".format(get_config('seed')))

All the preprocessing configurations and experiment settings/parameters are passed into the `setup` function. To see all available parameters, check the docstring:

In [ ]:
# help(setup)

In [ ]:
# init setup fold_strategy = expanding
s3 = []
for i in range(len(cut_sub_dfs)):
    s3.append(setup(
        cut_sub_dfs[i], 
        target = 'rolling_mean_grouped_soil', 
        #use_gpu = True, 
        fh = 50, 
        session_id = 123,
        fold_strategy = 'expanding', 
        numeric_imputation_target = 'drift', 
        ignore_features = ['Time', 'orig_index', 'gradient']
    ))

## ✅ Compare Models
This function trains and evaluates the performance of all estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
best_comp = []
for i in range(len(s3)):
    best_comp.append(s3[i].compare_models(include=['lr_cds_dt', 'br_cds_dt', 'ridge_cds_dt', 
                 'huber_cds_dt', 'auto_arima', 'knn_cds_dt', 'catboost_cds_dt']))

In [ ]:
# compare the ones without turbo => same models, useless
#best_comp_no_turbo = []
#for i in range(len(s3)):
#    best_comp_no_turbo.append(s3[i].compare_models(turbo=False))

`compare_models` by default uses all the estimators in model library (all except models with `Turbo=False`) . To see all available models you can use the function `models()`

In [ ]:
# check available models
models()

You can use the `include` and `exclude` parameter in the `compare_models` to train only select model or exclude specific models from training by passing the model id's in `exclude` parameter.

In [ ]:
#compare_ts_models = compare_models(include = ['ets', 'arima', 'theta', 'naive', 'snaive', 'grand_means', 'polytrend'])

In [ ]:
#compare_ts_models

The function above has return trained model object as an output. The scoring grid is only displayed and not returned. If you need access to the scoring grid you can use `pull` function to access the dataframe.

In [ ]:
#compare_ts_models_results = pull()
#compare_ts_models_results

By default `compare_models` return the single best performing model based on the metric defined in the `sort` parameter. Let's change our code to return 3 top models based on `MAE`.

In [ ]:
best_mae_models_top3 = []
for i in range(len(s3)):
    best_mae_models_top3.append(s3[i].compare_models(sort = 'MAE', n_select = 3,include=['lr_cds_dt', 'br_cds_dt', 'ridge_cds_dt', 
                 'huber_cds_dt', 'auto_arima', 'knn_cds_dt', 'catboost_cds_dt']))

In [ ]:
# list of top 3 models by MAE
for i in range(len(best_mae_models_top3)):
    print("This is model",i,":")
    print(best_mae_models_top3[i])

Some other parameters that you might find very useful in `compare_models` are:

- fold
- cross_validation
- budget_time
- errors
- parallel
- engine

You can check the docstring of the function for more info.

In [ ]:
# help(compare_models)

## ✅  Check Stats
The `check_stats` function is used to get summary statistics and run statistical tests on the original data or model residuals.

In [ ]:
# check stats on original data, swap index of exp to see stats-> otherwise use print->ugly
s3[0].check_stats()

In [ ]:
# check_stats on residuals of best model
for i in range(len(s3)):
    s3[i].check_stats(estimator = best_mae_models_top3[i][0])

## ✅ Experiment Logging
PyCaret integrates with many different type of experiment loggers (default = 'mlflow'). To turn on experiment tracking in PyCaret you can set `log_experiment` and `experiment_name` parameter. It will automatically track all the metrics, hyperparameters, and artifacts based on the defined logger.

In [ ]:
# from pycaret.time_series import *
# s = setup(data, fh = 3, session_id = 123, log_experiment='mlflow', experiment_name='airline_experiment')

In [ ]:
# compare models
# best = compare_models()

In [ ]:
# start mlflow server on localhost:5000
# !mlflow ui

By default PyCaret uses `MLFlow` logger that can be changed using `log_experiment` parameter. Following loggers are available:
    
    - mlflow
    - wandb
    - comet_ml
    - dagshub
    
Other logging related parameters that you may find useful are:

- experiment_custom_tags
- log_plots
- log_data
- log_profile

For more information check out the docstring of the `setup` function.

In [ ]:
# help(setup)

## ✅ Create Model
This function trains and evaluates the performance of a given estimator using cross-validation. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function. All the available models can be accessed using the models function.

In [ ]:
# check all the available models
models()

In [ ]:
exp[0]

In [ ]:
# train ets with default fold=3
best_models_mae = []
for i in range(len(s3)):
    print("Best model: ",i,": ",best_mae_models_top3[i][0])
    best_models_mae.append(exp[i].create_model(best_mae_models_top3[i][0], fold=3))

The function above has return trained model object as an output. The scoring grid is only displayed and not returned. If you need access to the scoring grid you can use `pull` function to access the dataframe.

In [ ]:
for i in range(len(best_models_mae)):
    ets_results = exp[i].pull()
    print("Model ",i,":",type(ets_results))
    print(ets_results)

In [ ]:
# train theta model with fold=5
#theta = create_model('theta', fold=5)

In [ ]:
# train theta with specific model parameters
#create_model('theta', deseasonalize = False, fold=5)

Some other parameters that you might find very useful in `create_model` are:

- cross_validation
- engine
- fit_kwargs

You can check the docstring of the function for more info.

In [ ]:
# help(create_model)

## ✅ Tune Model

The `tune_model` function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter. Metrics evaluated during cross-validation can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# train a dt model with default params
#dt = create_model('dt_cds_dt')

In [ ]:
# tune hyperparameters of dt
tuned_best_models = []
for i in range(len(best_models_mae)):
    print("This is for the",i,"model:",best_models_mae[i])
    tuned_best_models = tune_model(best_models_mae[i])

Metric to optimize can be defined in `optimize` parameter (default = 'MASE'). Also, a custom tuned grid can be passed with `custom_grid` parameter. 

In [ ]:
for i in range(len(best_models_mae)):
    print(tuned_best_models[i])

In [ ]:
current = best[1][0]
print(str(current.regressor), type(str(current.regressor)))
current.get_params().keys()


In [ ]:
# define tuning grid -> no grid for model defined, TODO: come back later
#grid = {'regressor__max_depth' : [None, 2, 4, 6, 8, 10, 12]}
#sp = {'regressor__max_depth' : [None, 2, 4, 6, 8, 10, 12]}
# define the custom grid
param_grid_linear = {
    'degree': [1, 2, 3],
    'deseasonal_model': ['multiplicative', 'additive'],
    #'fe_target_rr': [True, False],
    #'regressor__copy_X': [True, False],
    #'regressor__fit_intercept': [True, False],
    'regressor__n_jobs': [-1, 1, 2],
    #'regressor__normalize': [True, False],
    'regressor__positive': [True, False],
    'sp': [1, 2, 3],
    'window_length': [10, 20, 30]
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}

# tune model with custom grid and metric = MAE
tuned_best_models_grid = []
for i in range(len(exp)):
    print("This is for the",i,"model:",best[i][0])
    if str(best[i][0].regressor) == "LinearRegression(n_jobs=-1)":
        tuned_best_models_grid.append(exp[i].tune_model(best[i][0], custom_grid = param_grid_linear, optimize = 'MAE'))
    elif str(best[i][0].regressor) == "KNeighborsRegressor(n_jobs=-1)":
        tuned_best_models_grid.append(exp[i].tune_model(best[i][0], custom_grid = param_grid_knn, optimize = 'MAE'))

In [ ]:
# see tuned_dt params
for i in range(len(best_models)):
    print(tuned_best_models_grid[i])

In [ ]:
# to access the tuner object you can set return_tuner = True
tuners = []
tuned_best_models_tuner = []

for i in range(len(best_models)):
    tuned_model, tuner = tune_model(best_models[i], custom_grid = param_grid, return_tuner=True) #tuned_best_models_grid[i] does not work
    tuned_best_models_tuner.append(tuned_model)
    tuners.append(tuner)    

In [ ]:
# model object
for i in range(len(tuned_best_models_tuner)):
    tuned_best_models_tuner[i]

In [ ]:
# tuner object
for i in range(len(best_models)):
    tuner[i]

For more details on all available `search_library` and `search_algorithm` please check the docstring. Some other parameters that you might find very useful in `tune_model` are:

- choose_better
- custom_scorer
- n_iter
- search_algorithm
- optimize

You can check the docstring of the function for more info.

In [ ]:
# help(tune_model)

## ✅ Blend Models

In [ ]:
len(tuned_best_models_grid)

This function trains a `EnsembleForecaster` for select models passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# top 3 models based on mae
for i in range(len(best)):
    print(tuned_best_models_grid[i])

In [ ]:
# blend top 3 models
blended_model = []
for i in range(len(best_models)):
    print(tuned_best_models_grid[i])
    blended_model.append(blend_models(best_models))

Some other parameters that you might find very useful in `blend_models` are:

- choose_better
- method
- weights
- fit_kwargs
- optimize

You can check the docstring of the function for more info.

In [ ]:
# help(blend_models)

## ✅ Plot Model

This function analyzes the performance of a trained model on the hold-out set. It may require re-training the model in certain cases.

In [ ]:
blended_model[0][0]

In [ ]:
# extend forecast horizon

In [ ]:
# plot forecast
for i in range(len(best_models)):
    exp[i].plot_model(blended_model[i][0], plot = 'forecast', data_kwargs = {'fh' : 500}) #exp[i].plot_model(best[i], plot = 'forecast', data_kwargs = {'fh' : 500})

In [ ]:
future_df = exp[0].predict_model(best[0], fh=144)
print(future_df)

In [ ]:
cut_sub_dfs[0]['Time'][len(cut_sub_dfs[0])-1]

In [ ]:
cut_sub_dfs[0].tail()

In [ ]:
len(cut_sub_dfs)

In [ ]:
# TODO: for loop
all_dates = []
for i in range(len(cut_sub_dfs)):
    end = cut_sub_dfs[i]['Time'][len(cut_sub_dfs[i])-1]
    #print("end before adding: ", end)
    end=end+pd.Timedelta(days=2)
    #print("end after adding: ", end)
    all_dates.append(pd.date_range(start=cut_sub_dfs[i]['Time'][0], end=end, freq='10T')) #end= cut_sub_dfs[1]['Time'][len(cut_sub_dfs[1])-1]
    print("Dataframe",i,":",all_dates[i])

In [ ]:
# create new data (only dates)
new_data = []
for i in range(len(cut_sub_dfs)):
    new_data.append(pd.DataFrame())
    new_data[i]['Time'] = all_dates[i]
    new_data[i]['hour'] = [i.hour for i in new_data[i]['Time']]
    new_data[i]['minute'] = [i.minute for i in new_data[i]['Time']]
    new_data[i]['date'] = [i.day for i in new_data[i]['Time']]
    new_data[i]['month'] = [i.month for i in new_data[i]['Time']]
    
    print("Dataframe",i,":")
    print(new_data[i].head(1))

In [ ]:
new_data[0].head()

In [ ]:
future_df = []
for i in range(len(cut_sub_dfs)):
    future_df.append(exp[i].predict_model(best_models[i], X=new_data[i]))

In [ ]:
for i in range(len(cut_sub_dfs)):
    future_df[i].plot()

In [ ]:
# plot acf
# for certain plots you don't need a trained model
plot_model(plot = 'acf')

In [ ]:
# plot diagnostics
# for certain plots you don't need a trained model
plot_model(plot = 'diagnostics')

Some other parameters that you might find very useful in `plot_model` are:

- fig_kwargs
- data_kwargs
- display_format
- return_fig
- return_data
- save

You can check the docstring of the function for more info.

In [ ]:
# help(plot_model)

## ✅ Finalize Model
This function trains a given model on the entire dataset including the hold-out set.

In [ ]:
final_best = finalize_model(best) #TODO: for loop

In [ ]:
final_best

## ✅ Deploy Model
This function deploys the entire ML pipeline on the cloud.

**AWS:**  When deploying model on AWS S3, environment variables must be configured using the command-line interface. To configure AWS environment variables, type `aws configure` in terminal. The following information is required which can be generated using the Identity and Access Management (IAM) portal of your amazon console account:

- AWS Access Key ID
- AWS Secret Key Access
- Default Region Name (can be seen under Global settings on your AWS console)
- Default output format (must be left blank)

**GCP:** To deploy a model on Google Cloud Platform ('gcp'), the project must be created using the command-line or GCP console. Once the project is created, you must create a service account and download the service account key as a JSON file to set environment variables in your local environment. Learn more about it: https://cloud.google.com/docs/authentication/production

**Azure:** To deploy a model on Microsoft Azure ('azure'), environment variables for the connection string must be set in your local environment. Go to settings of storage account on Azure portal to access the connection string required.
AZURE_STORAGE_CONNECTION_STRING (required as environment variable)
Learn more about it: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?toc=%2Fpython%2Fazure%2FTOC.json

In [ ]:
# deploy model on aws s3
# deploy_model(best, model_name = 'my_first_platform_on_aws',
#             platform = 'aws', authentication = {'bucket' : 'pycaret-test'})

In [ ]:
# load model from aws s3
# loaded_from_aws = load_model(model_name = 'my_first_platform_on_aws', platform = 'aws',
#                              authentication = {'bucket' : 'pycaret-test'})

# loaded_from_aws

## ✅ Save / Load Model
This function saves the transformation pipeline and a trained model object into the current working directory as a pickle file for later use.

In [ ]:
# save model
for i in range(len(best_models)):
    save_model(best_models[i], 'my_first_SPLIT_model' + str(i))

In [ ]:
# load model
for i in range(len(best_models)): # will not work need to know how many models->use os TODO
    loaded_from_disk = load_model('my_first_SPILT_model'+str(i))
    loaded_from_disk

## ✅ Save / Load Experiment
This function saves all the experiment variables on disk, allowing to later resume without rerunning the setup function.

In [ ]:
# save experiment
save_experiment('my_SPLIT_experiment')

In [ ]:
# load experiment from disk
exp_from_disk = load_experiment('my_SPLIT_experiment', data=data)